* [1. Problem Statement](#0)
* [2. Getting Started - Load Libraries and Dataset](#1)
    * [2.1. Load Libraries](#1.1)    
    * [2.2. Load Dataset](#1.2)

<a id='0'></a>
# 1. Problem Statement

link: https://github.com/RomelTorres/alpha_vantage

In [ ]:
https://medium.com/codex/alpha-vantage-an-introduction-to-a-highly-efficient-free-stock-api-6d17f4481bf

In [ ]:
Content:
    Stock
    Many stocks/Indices
    Crypto
    FX
    Finacial
    Real Time Data
    List of tickers available 

In [ ]:
from alpha_vantage.timeseries import TimeSeries
import pandas as pd
import time

In [ ]:
# API Key
key = "AAUUBAYEDCI353AC"

In [ ]:
ts = TimeSeries(key=key, output_format='pandas')
data = ts.get_daily(symbol='TSLA', outputsize='full')[0]
data.columns = ["open","high","low","close","volume"]
data = data.iloc[::-1]

In [ ]:
data

## Extracting stock data (historical close price) for multiple stocks


In [ ]:
all_tickers = ["AAPL","MSFT","CSCO","AMZN","GOOG",
               "FB","BA","MMM","XOM","NKE","INTC"]
close_prices = pd.DataFrame()
api_call_count = 1
ts = TimeSeries(key=key, output_format='pandas')
start_time = time.time()
for ticker in all_tickers:
    data = ts.get_intraday(symbol=ticker,interval='1min', outputsize='compact')[0]
    api_call_count+=1
    data.columns = ["open","high","low","close","volume"]
    data = data.iloc[::-1]
    close_prices[ticker] = data["close"]
    if api_call_count==5:
        api_call_count = 1
        time.sleep(60 - ((time.time() - start_time) % 60.0))

In [ ]:
close_prices

## Extracting OHLCV data for multiple stocks


In [ ]:
all_tickers = ["AAPL","MSFT","CSCO","AMZN","GOOG",
               "FB","BA","MMM","XOM","NKE","INTC"]
ohlv_dict = {}
api_call_count = 1
ts = TimeSeries(key=open(key_path,'r').read(), output_format='pandas')
start_time = time.time()
for ticker in all_tickers:
    data = ts.get_intraday(symbol=ticker,interval='1min', outputsize='compact')[0]
    api_call_count+=1
    data.columns = ["open","high","low","close","volume"]
    data = data.iloc[::-1]
    ohlv_dict[ticker] = data
    if api_call_count==5:
        api_call_count = 1
        time.sleep(60 - ((time.time() - start_time) % 60.0))

In [ ]:
ohlv_dict['AMZN']